# Small demo for PICO extraction and text summarization with LLMs

Models selected: BioELECTRA-PICO for PIO extraction. C extraction is implemented as a manual search for frequent comparators (e.g. placebo). Extractive summarization is implemented with TextRank algorithm.

In [8]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1" # suppresses Huggingface warning of storing data rather than symlinking it
from transformers import AutoTokenizer, AutoModelForPreTraining, pipeline

import sys
sys.path.append('../../')
# load functions for import of clinicaltrials.gov data written previously
from app.data.loader import load_trials_json, extract_from_clinicaltrials
from app.nlp.utils import * # custom functions required for NER and summarization
from app.nlp.evaluate_model import elements_from_cell, substring_partial_overlap, add_partial_overlap_cols

In [2]:
# Load and process data from clinicaltrials.gov
PROJECT_ROOT = 'c:\\Users\\elena.jolkver\\Documents\\github\\biomed_extractor'

# Data directory at top level
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')

df_json = load_trials_json(filepath = DATA_DIR, filename ='example_trials.json')
#print(df_json.head())
mydf = extract_from_clinicaltrials(df_json)
mydf.head()

Loaded 100 records from example_trials.json


,nctId,briefSummary,detailedDescription,inclusion_criteria,exclusion_criteria,intervention_name_clean,outcomes_name
0,NCT00105105,The purpose of this study is to evaluate the e...,"This will be a double blind, placebo controlle...",diagnosis of alzheimer's disease; women must h...,women with an intact uterus; a clinically sign...,Mifepristone,effects on cognition
1,NCT00160147,This is a 10-week study with bifeprunox and pl...,,diagnosis of dementia of the alzheimer's type,history of seizure disorder; clinically signif...,bifeprunox; Placebo,Brief Psychiatric Rating Scale (BPRS) Total Score
2,NCT00299988,The overall goal of this double-blind Phase II...,Abnormal processing of the beta-amyloid protei...,1. diagnosis of probable alzheimer's disease (...,1. non-alzheimer dementia.2. active renal dise...,Intravenous Immunoglobulin; Placebo,ADAS-Cog; CGIC
3,NCT00334568,Clinical features in patients with the familia...,,meets the national institute of neurological a...,has a history of or suffers from claustrophobi...,Rosiglitazone XR (extended release) oral table...,Change in global and regional cerebral glucose...
4,NCT00362024,MK0952 is a phosphodiesterase type IV (PDE4) i...,,"male or females; age \>/= 55 years, with mild-...",patients must not be living in nursing home or...,MK0952; Comparator: Placebo,


In [3]:
# Load BioM-ELECTRA-Base-PICO model
# TODO: clean up C:\Users\USER\.cache\huggingface\hub, deleting unneeded models
#tokenizer = AutoTokenizer.from_pretrained("kamalkraj/BioELECTRA-PICO")
#model = AutoModelForPreTraining.from_pretrained("kamalkraj/BioELECTRA-PICO")
ner_pipeline = pipeline("token-classification", 
                        model="kamalkraj/BioELECTRA-PICO",
                        aggregation_strategy="simple")

Device set to use cpu


In [4]:
ner_res = process_trials_for_PICO(mydf, ner_pipeline)
ner_res.head()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,nctId,briefSummary,detailedDescription,inclusion_criteria,exclusion_criteria,intervention_name_clean,outcomes_name,population_extracted,intervention_extracted,outcome_extracted,comparator_extracted,summary_extracted
0,NCT00105105,The purpose of this study is to evaluate the e...,"This will be a double blind, placebo controlle...",diagnosis of alzheimer's disease; women must h...,women with an intact uterus; a clinically sign...,Mifepristone,effects on cognition,able to provide written informed consent; ambu...,1073; acetylcholinesterase; aricept exelon; re...,"cognition and behavior, depression, safety, as...",placebo,The purpose of this study is to evaluate the e...
1,NCT00160147,This is a 10-week study with bifeprunox and pl...,,diagnosis of dementia of the alzheimer's type,history of seizure disorder; clinically signif...,bifeprunox; Placebo,Brief Psychiatric Rating Scale (BPRS) Total Score,dementia of the alzheimer ' s type,bifeprunox,,,This is a 10-week study with bifeprunox and pl...
2,NCT00299988,The overall goal of this double-blind Phase II...,Abnormal processing of the beta-amyloid protei...,1. diagnosis of probable alzheimer's disease (...,1. non-alzheimer dementia.2. active renal dise...,Intravenous Immunoglobulin; Placebo,ADAS-Cog; CGIC,. 4. an able caregiver willing to participate ...,intravenous immunoglobulin; ivig,"cognitive, behavioral and functional measures;...",placebo,The overall goal of this double-blind Phase II...
3,NCT00334568,Clinical features in patients with the familia...,,meets the national institute of neurological a...,has a history of or suffers from claustrophobi...,Rosiglitazone XR (extended release) oral table...,Change in global and regional cerebral glucose...,and have not been confirmed as post - menopau...,,,,Clinical features in patients with the familia...
4,NCT00362024,MK0952 is a phosphodiesterase type IV (PDE4) i...,,"male or females; age \>/= 55 years, with mild-...",patients must not be living in nursing home or...,MK0952; Comparator: Placebo,,"age \ > / = 55 years, with mild - to - moderat...",mk0952; phosphodiesterase,,,MK0952 is a phosphodiesterase type IV (PDE4) i...


## Evaluate model results on GOLD standard

In [5]:
# load gold standard trials
# Load and process data from clinicaltrials.gov
PROJECT_ROOT = 'c:\\Users\\elena.jolkver\\Documents\\github\\biomed_extractor'

# Data directory at top level
DATA_DIR = os.path.join(PROJECT_ROOT, 'data\\annotated')

df_json = load_trials_json(filepath = DATA_DIR, filename ='ctg-studies_for_gold.json')
#print(df_json.head())
mydf_manual_annotation = extract_from_clinicaltrials(df_json)
mydf_manual_annotation.head()

Loaded 20 records from ctg-studies_for_gold.json


,nctId,briefSummary,detailedDescription,inclusion_criteria,exclusion_criteria,intervention_name_clean,outcomes_name
0,NCT00667810,This is a study to evaluate the efficacy and s...,,"diagnosis of probable alzheimer disease (ad), ...",significant neurological disease other than ad...,bapineuzumab; placebo,The Change From Baseline in the Alzheimer's Di...
1,NCT00676143,This is a study to evaluate the efficacy and s...,,"diagnosis of probable ad, with mmse score of 1...",significant neurological disease other than ad...,bapineuzumab; placebo,Change From Baseline in Alzheimer's Disease As...
2,NCT01561430,The purpose of this Phase 1/Phase 2 study is t...,,meets criteria for mci due to ad or mild adall...,participant in another drug or device study; h...,LY2886721; Placebo,Change From Baseline to 12 Weeks in Cerebrospi...
3,NCT01900665,To test the idea that solanezumab will slow th...,,meets national institute of neurological and c...,does not have a reliable caregiver who is in f...,Solanezumab; Placebo,Change From Baseline in Alzheimer's Disease As...
4,NCT02565511,The purpose of this study was to test whether ...,The study (also known as the Generation Study ...,consented to receive disclosure of their risk ...,any disability that prevented the participant ...,CAD106 Immunotherapy; Placebo to CAD106; CNP52...,Time to Event (Diagnosis of Mild Cognitive Imp...


In [6]:
ner_res_model = process_trials_for_PICO(mydf_manual_annotation, ner_pipeline)
ner_res_model.sort_values(by=['nctId'], inplace=True)
# rename columns to match gold standard
ner_res_model.rename(columns={'population_extracted': 'population',
                             'intervention_extracted': 'intervention',
                             'comparator_extracted': 'comparator',
                                'outcome_extracted': 'outcome',
                                'summary_extracted': 'summary'}, inplace=True)
                                
ner_res_model.head()

,nctId,briefSummary,detailedDescription,inclusion_criteria,exclusion_criteria,intervention_name_clean,outcomes_name,population,intervention,outcome,comparator,summary
0,NCT00667810,This is a study to evaluate the efficacy and s...,,"diagnosis of probable alzheimer disease (ad), ...",significant neurological disease other than ad...,bapineuzumab; placebo,The Change From Baseline in the Alzheimer's Di...,diagnosis of probable alzheimer disease ( ad )...,bapineuzumab,efficacy; safety,,This is a study to evaluate the efficacy and s...
1,NCT00676143,This is a study to evaluate the efficacy and s...,,"diagnosis of probable ad, with mmse score of 1...",significant neurological disease other than ad...,bapineuzumab; placebo,Change From Baseline in Alzheimer's Disease As...,"diagnosis of probable ad, with mmse score of 1...",bapineuzumab,efficacy; safety,,This is a study to evaluate the efficacy and s...
2,NCT01561430,The purpose of this Phase 1/Phase 2 study is t...,,meets criteria for mci due to ad or mild adall...,participant in another drug or device study; h...,LY2886721; Placebo,Change From Baseline to 12 Weeks in Cerebrospi...,absence of dementia; clinical dementia rating ...,,,,The purpose of this Phase 1/Phase 2 study is t...
3,NCT01900665,To test the idea that solanezumab will slow th...,,meets national institute of neurological and c...,does not have a reliable caregiver who is in f...,Solanezumab; Placebo,Change From Baseline in Alzheimer's Disease As...,) result consistent with the presence of amylo...,solanezumab,cognitive decline,placebo,To test the idea that solanezumab will slow th...
4,NCT02565511,The purpose of this study was to test whether ...,The study (also known as the Generation Study ...,consented to receive disclosure of their risk ...,any disability that prevented the participant ...,CAD106 Immunotherapy; Placebo to CAD106; CNP52...,Time to Event (Diagnosis of Mild Cognitive Imp...,"homozygous apoe4 genotype.; male or female, ag...",cad106; cnp520,"and body weight loss; cognitive function, brai...",placebo,The purpose of this study was to test whether ...


In [7]:
# load GOLD-standard results
df_gold = load_trials_json(filepath = DATA_DIR, filename ='gold_standard.json')
for col in ["population", "intervention","comparator", "outcome"]:
    df_gold[col] = df_gold[col].apply(
        lambda x: '; '.join(str(e) for e in x) if isinstance(x, list) else x
    )
    # set to lowercase (to align with automatic extraction)
    df_gold[col] = df_gold[col].str.lower()
df_gold.sort_values(by=['doc_id'], inplace=True)
df_gold["intervention"] = df_gold["intervention"].apply(normalize_intervention)
df_gold

Loaded 20 records from gold_standard.json


,doc_id,population,intervention,comparator,outcome
12,NCT00667810,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,the change from baseline in the alzheimer's di...
11,NCT00676143,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,change from baseline in alzheimer's disease as...
13,NCT01561430,participants with mild cognitive impairment (m...,ly2886721,placebo,change from baseline to 12 weeks in cerebrospi...
19,NCT01900665,participants with mild ad,solanezumab,placebo,change from baseline in alzheimer's disease as...
3,NCT02565511,cognitively unimpaired apoe4 homozygotes (hms)...,cad106 immunotherapy cnp520 alum,placebo,time to event (diagnosis of mild cognitive imp...
7,NCT02769000,patients with early alzheimer's dementia,radiation 10 gy radiation 20 gy,,ctcae toxicity grading
14,NCT02783573,participants with mild alzheimer's disease (ad...,lanabecestat,placebo,change from baseline in alzheimer´s disease as...
9,NCT02791191,participants with mild alzheimer's disease (ad...,ly3202626,placebo,change from baseline in f-av-1451 positron emi...
15,NCT02884492,elderly subjects with different clinical and b...,18f thk 5351,,18f-thk-5351 standardized uptake value ratio
10,NCT02972658,participants with early alzheimer's disease de...,lanabecestat,,change from baseline analysis on the 13-item a...


In [9]:
pico_cols = ["population", "intervention", "comparator", "outcome"]
df_gold_with_partial, evaluation_table = add_partial_overlap_cols(df_gold.copy(), ner_res_model, pico_cols)
df_gold_with_partial

,doc_id,population,intervention,comparator,outcome,population_partial_precision,population_partial_recall,population_partial_f1,intervention_partial_precision,intervention_partial_recall,intervention_partial_f1,comparator_partial_precision,comparator_partial_recall,comparator_partial_f1,outcome_partial_precision,outcome_partial_recall,outcome_partial_f1
12,NCT00667810,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,the change from baseline in the alzheimer's di...,0.125000,0.428571,0.193548,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.500000,0.055556,0.100000
11,NCT00676143,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,change from baseline in alzheimer's disease as...,0.066667,0.142857,0.090909,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.500000,0.058824,0.105263
13,NCT01561430,participants with mild cognitive impairment (m...,ly2886721,placebo,change from baseline to 12 weeks in cerebrospi...,0.495327,0.950000,0.651148,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
19,NCT01900665,participants with mild ad,solanezumab,placebo,change from baseline in alzheimer's disease as...,0.100000,0.750000,0.176471,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.153846,0.266667
3,NCT02565511,cognitively unimpaired apoe4 homozygotes (hms)...,cad106 immunotherapy cnp520 alum,placebo,time to event (diagnosis of mild cognitive imp...,0.250000,0.800000,0.380952,1.000000,0.500000,0.666667,1.0,1.000000,1.0,0.250000,0.133333,0.173913
7,NCT02769000,patients with early alzheimer's dementia,radiation 10 gy radiation 20 gy,,ctcae toxicity grading,0.450980,1.000000,0.621622,0.121212,0.500000,0.195122,1.0,1.000000,1.0,0.058824,0.666667,0.108108
14,NCT02783573,participants with mild alzheimer's disease (ad...,lanabecestat,placebo,change from baseline in alzheimer´s disease as...,0.392857,1.000000,0.564103,0.500000,1.000000,0.666667,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9,NCT02791191,participants with mild alzheimer's disease (ad...,ly3202626,placebo,change from baseline in f-av-1451 positron emi...,0.489796,1.000000,0.657534,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
15,NCT02884492,elderly subjects with different clinical and b...,18f thk 5351,,18f-thk-5351 standardized uptake value ratio,0.444444,0.769231,0.563380,0.428571,1.000000,0.600000,1.0,1.000000,1.0,0.000000,0.000000,0.000000
10,NCT02972658,participants with early alzheimer's disease de...,lanabecestat,,change from baseline analysis on the 13-item a...,0.176471,0.285714,0.218182,0.333333,1.000000,0.500000,0.0,0.000000,0.0,1.000000,0.750000,0.857143


In [10]:
evaluation_table

,precision,recall,f1
population,0.378636,0.722026,0.477297
intervention,0.794156,0.829167,0.759756
comparator,0.600000,0.566667,0.575000
outcome,0.362562,0.149262,0.155473


In [ ]:
# combine results and gold standard
ner_res_model.rename(columns={'nctId': 'doc_id'}, inplace=True)
ner_res_model = ner_res_model[["doc_id","population", "intervention","comparator", "outcome"]].copy()
df_side_by_side = ner_res_model.merge(df_gold_with_partial, on='doc_id', how='outer', suffixes=('_extracted', '_gold'))
df_side_by_side

,doc_id,population_extracted,intervention_extracted,comparator_extracted,outcome_extracted,population_gold,intervention_gold,comparator_gold,outcome_gold,population_partial_precision,...,population_partial_f1,intervention_partial_precision,intervention_partial_recall,intervention_partial_f1,comparator_partial_precision,comparator_partial_recall,comparator_partial_f1,outcome_partial_precision,outcome_partial_recall,outcome_partial_f1
0,NCT00667810,diagnosis of probable alzheimer disease ( ad )...,bapineuzumab,,efficacy; safety,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,the change from baseline in the alzheimer's di...,0.125000,...,0.193548,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.500000,0.055556,0.100000
1,NCT00676143,"diagnosis of probable ad, with mmse score of 1...",bapineuzumab,,efficacy; safety,patients with mild to moderate alzheimer disease,bapineuzumab,placebo,change from baseline in alzheimer's disease as...,0.066667,...,0.090909,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.500000,0.058824,0.105263
2,NCT01561430,absence of dementia; clinical dementia rating ...,,,,participants with mild cognitive impairment (m...,ly2886721,placebo,change from baseline to 12 weeks in cerebrospi...,0.495327,...,0.651148,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,NCT01900665,) result consistent with the presence of amylo...,solanezumab,placebo,cognitive decline,participants with mild ad,solanezumab,placebo,change from baseline in alzheimer's disease as...,0.100000,...,0.176471,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,0.153846,0.266667
4,NCT02565511,"homozygous apoe4 genotype.; male or female, ag...",cad106; cnp520,placebo,"and body weight loss; cognitive function, brai...",cognitively unimpaired apoe4 homozygotes (hms)...,cad106 immunotherapy cnp520 alum,placebo,time to event (diagnosis of mild cognitive imp...,0.250000,...,0.380952,1.000000,0.500000,0.666667,1.0,1.000000,1.0,0.250000,0.133333,0.173913
5,NCT02769000,able to complete neurocognitive function asses...,beam irradiation; bnt; chemotherapy; external ...,,quality of life scores; alzheimer ' s and deme...,patients with early alzheimer's dementia,radiation 10 gy radiation 20 gy,,ctcae toxicity grading,0.450980,...,0.621622,0.121212,0.500000,0.195122,1.0,1.000000,1.0,0.058824,0.666667,0.108108
6,NCT02783573,evidence of amyloid pathology.; for a diagnosi...,as lanabecestat,,,participants with mild alzheimer's disease (ad...,lanabecestat,placebo,change from baseline in alzheimer´s disease as...,0.392857,...,0.564103,0.500000,1.000000,0.666667,0.0,0.000000,0.0,0.000000,0.000000,0.000000
7,NCT02791191,mini - mental state examination score of 20 to...,ly3202626,,safety,participants with mild alzheimer's disease (ad...,ly3202626,placebo,change from baseline in f-av-1451 positron emi...,0.489796,...,0.657534,1.000000,1.000000,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
8,NCT02884492,1. age 50 and older. 2. meet criteria for eith...,18f thk 5351; brain mri; neuropsychological; pet,,,elderly subjects with different clinical and b...,18f thk 5351,,18f-thk-5351 standardized uptake value ratio,0.444444,...,0.563380,0.428571,1.000000,0.600000,1.0,1.000000,1.0,0.000000,0.000000,0.000000
9,NCT02972658,participants previously enrolled in amaranth (...,azes; lanabecestat; ly3314814,placebo,adas - cog; alzheimer ' s disease assessment s...,participants with early alzheimer's disease de...,lanabecestat,,change from baseline analysis on the 13-item a...,0.176471,...,0.218182,0.333333,1.000000,0.500000,0.0,0.000000,0.0,1.000000,0.750000,0.857143


In [14]:
table = pd.DataFrame(
    {
        "precision":   [df_gold_with_partial[f"{col}_partial_precision"].mean() for col in pico_cols],
        "recall":      [df_gold_with_partial[f"{col}_partial_recall"].mean()    for col in pico_cols],
        "f1":          [df_gold_with_partial[f"{col}_partial_f1"].mean()        for col in pico_cols]
    },
    index=[col.replace("_extracted", "") for col in pico_cols]
)
print(table)

              precision    recall        f1
population     0.378636  0.722026  0.477297
intervention   0.794156  0.829167  0.759756
comparator     0.600000  0.566667  0.575000
outcome        0.362562  0.149262  0.155473
